In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('./LiberalArtsEmployment.csv')
df = df.rename(columns={
    "Institution Name": "Institution",
    "Sector of institution (HD2023)": "Sector"
})
df = df.drop(columns = ["Unnamed: 63"])
df.head()

,UnitID,Institution,Sector,All ranks (S2023_SIS All full-time instructional staff),Full-time non-instructional staff - number (SAL2023_NIS),Management - number (SAL2023_NIS),Business and Financial Operations - number (SAL2023_NIS),Grand total (EF2023 All students total),All ranks (S2022_SIS_RV All full-time instructional staff),Full-time non-instructional staff - number (SAL2022_NIS_RV),...,All ranks (S2013_SIS_RV All full-time instructional staff),Full-time non-instructional staff - number (SAL2013_NIS_RV),Management - number (SAL2013_NIS_RV),Business and Financial Operations - number (SAL2013_NIS_RV),Grand total (EF2013_RV All students total),All ranks (S2012_SIS_RV All full-time instructional staff),Full-time non-instructional staff - number (SAL2012_NIS_RV),Management - number (SAL2012_NIS_RV),Business and Financial Operations - number (SAL2012_NIS_RV),Grand total (EF2012_RV All students total)
0,138600,Agnes Scott College,2,95.0,160.0,25.0,25.0,1077.0,93.0,214,...,75.0,215.0,34.0,26.0,915.0,70.0,206.0,36.0,25.0,885.0
1,168546,Albion College,2,117.0,249.0,55.0,26.0,1354.0,124.0,260,...,109.0,243.0,51.0,16.0,1307.0,115.0,238.0,51.0,20.0,1382.0
2,210571,Albright College,2,110.0,218.0,34.0,35.0,1652.0,114.0,203,...,119.0,241.0,41.0,12.0,2384.0,118.0,244.0,72.0,9.0,2264.0
3,210669,Allegheny College,2,133.0,236.0,83.0,11.0,1231.0,140.0,246,...,183.0,273.0,52.0,22.0,2161.0,168.0,273.0,45.0,24.0,2140.0
4,217624,Allen University,2,35.0,94.0,12.0,4.0,677.0,42.0,64,...,44.0,55.0,15.0,1.0,651.0,44.0,75.0,25.0,1.0,672.0


In [3]:
id_vars = ["UnitID", "Institution", "Sector"]
long_df = df.melt(id_vars=id_vars, var_name="raw", value_name="Count")

In [7]:
def parse_year_and_type(raw):
    year_match = re.search(r"(20\d{2})", raw)
    year = int(year_match.group(1)) if year_match else None

    # Infer type
    raw_lower = raw.lower()
    if "non-instructional staff" in raw_lower:
        pop = "staff"
    elif "instructional staff" in raw_lower:
        pop = "faculty"
    elif "management" in raw_lower:
        pop = "academic_dmin"
    elif "business" in raw_lower:
        pop = "financial_admin"
    elif "students" in raw_lower:
        pop = "enrollment"
    else:
        pop = "other"

    return pd.Series([year, pop])

In [22]:
long_df[["Year", "Pop"]] = long_df["raw"].apply(parse_year_and_type)
long_df = long_df[['UnitID', 'Institution', 'Sector', 'Year', 'Pop', 'Count']].dropna()

In [23]:
long_df

,UnitID,Institution,Sector,Year,Pop,Count
0,138600,Agnes Scott College,2,2023,faculty,95.0
1,168546,Albion College,2,2023,faculty,117.0
2,210571,Albright College,2,2023,faculty,110.0
3,210669,Allegheny College,2,2023,faculty,133.0
4,217624,Allen University,2,2023,faculty,35.0
...,...,...,...,...,...,...
13195,210401,Willamette University,2,2012,enrollment,2931.0
13196,168342,Williams College,2,2012,enrollment,2124.0
13197,206525,Wittenberg University,2,2012,enrollment,1894.0
13198,218973,Wofford College,2,2012,enrollment,1619.0
